In [ ]:
#r "nuget:Microsoft.ML"

Installed Packages Microsoft.ML, 1.6.0

In [ ]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var trainDataPath = "../datasets/weatherAUS-train.csv";
var testDataPath = "../datasets/weatherAUS-test.csv";

In [ ]:
public class WeatherData
{   
    [LoadColumn(0, 12)]
    public float Features { get; set; }
    
    [LoadColumn(13)]
    public bool RainTomorrow { get; set; }
}

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
var trainingDataView = mlContext.Data.LoadFromTextFile<WeatherData>(trainDataPath, hasHeader: true, separatorChar: ',');
var testDataView = mlContext.Data.LoadFromTextFile<WeatherData>(testDataPath, hasHeader: true, separatorChar: ',');

In [ ]:
var dataProcessPipeline = mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "KeyColumn", inputColumnName: nameof(WeatherData.RainTomorrow))
                                                         .Append(mlContext.Transforms.Concatenate("Features", nameof(WeatherData.Features)))
                                                         .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"))
                                                         .AppendCacheCheckpoint(mlContext);

In [ ]:
var linearSvmTrainer = mlContext.BinaryClassification.Trainers.LinearSvm(labelColumnName: "KeyColumn", featureColumnName: "Features");
var trainer = mlContext.MulticlassClassification.Trainers.OneVersusAll(linearSvmTrainer, labelColumnName: "KeyColumn");

var trainingPipeline = dataProcessPipeline.Append(trainer);

In [ ]:
var trainedModel = trainingPipeline.Fit(trainingDataView);

In [ ]:
var predictions = trainedModel.Transform(testDataView);

In [ ]:
var metrics = mlContext.MulticlassClassification.Evaluate(data: predictions, labelColumnName: "KeyColumn", scoreColumnName: "Score");

In [ ]:
Console.WriteLine($"Micro Accuracy: {metrics.MicroAccuracy:P2}");
Console.WriteLine($"Macro Accuracy: {metrics.MacroAccuracy:P2}");
Console.WriteLine($"Log Loss: {metrics.LogLoss:F2}");
Console.WriteLine(metrics.ConfusionMatrix.GetFormattedConfusionTable());

Micro Accuracy: 84,57%
Macro Accuracy: 71,77%
Log Loss: 0,37

Confusion table
          ||======================
PREDICTED ||        0 |        1 | Recall
TRUTH     ||======================
        0 ||   17.627 |      937 | 0,9495
        1 ||    2.753 |    2.601 | 0,4858
          ||======================
Precision ||   0,8649 |   0,7352 |

